In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import abline_plot
import requests
import json
import pprint

In [4]:
df = pd.read_csv('Avoin_data_eduskuntavaalit_2019_valintatiedot.csv', delimiter=',')

In [5]:
df.head()

,vaalipiiri,puolue,"Valintatieto (0=ei valita, 1=valitaan, 2=varalla)","Suomen pitää olla edelläkävijä ilmastonmuutoksen vastaisessa taistelussa, vaikka se aiheuttaisi suomalaisille kustannuksia.",Suomen ei pidä kiirehtiä kieltämään uusien bensa- ja dieselautojen myyntiä.,Valtion pitää ohjata suomalaiset syömään vähemmän lihaa esimerkiksi verotuksen avulla.,Metsiä hakataan Suomessa liikaa.,"Kun valtion menoja ja tuloja tasapainotetaan, se on tehtävä mieluummin menoja karsimalla kuin veroja kiristämällä.","Sosiaaliturvaa tulee kehittää niin, että osa nykyisistä tuista korvataan kaikille työikäisille maksettavalla, vastikkeettomalla perustulolla.",Euron ulkopuolella Suomi pärjäisi paremmin.,...,Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa.,Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle.,Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan.,Lappi. Alkuperäiskansojen oikeuksia koskeva ILO-sopimus pitää ratifioida seuraavalla hallituskaudella.,Lappi. Jäämeren rata pitää rakentaa.,Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa..1,Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle..1,Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan..1,Lappi. Alkuperäiskansojen oikeuksia koskeva ILO-sopimus pitää ratifioida seuraavalla hallituskaudella. .1,Lappi. Jäämeren rata pitää rakentaa..1
0,Uudenmaan vaalipiiri,Kokoomus,0,2,4,4,1,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uudenmaan vaalipiiri,Suomen Kommunistinen Puolue,0,5,NaN,4,5,1,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Uudenmaan vaalipiiri,Perussuomalaiset,0,1,5,1,2,5,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Uudenmaan vaalipiiri,Perussuomalaiset,0,1,5,1,1,1,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uudenmaan vaalipiiri,Perussuomalaiset,1,1,5,1,2,5,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.shape

(2437, 212)

In [12]:
pprint.pprint(list(df.columns))

lle kustannuksia.',
 'Suomen ei pidä kiirehtiä kieltämään uusien bensa- ja dieselautojen myyntiä.',
 'Valtion pitää ohjata suomalaiset syömään vähemmän lihaa esimerkiksi '
 'verotuksen avulla.',
 'Metsiä hakataan Suomessa liikaa.',
 'Kun valtion menoja ja tuloja tasapainotetaan, se on tehtävä mieluummin '
 'menoja karsimalla kuin veroja kiristämällä.',
 'Sosiaaliturvaa tulee kehittää niin, että osa nykyisistä tuista korvataan '
 'kaikille työikäisille maksettavalla, vastikkeettomalla perustulolla.',
 'Euron ulkopuolella Suomi pärjäisi paremmin.',
 'Sosiaali- ja terveyspalvelut on tuotettava ensisijaisesti julkisina '
 'palveluina.',
 'Vanhustenhoidon ulkoistamista yksityisille toimijoille tulee lisätä.',
 'Parantumattomasti sairaalla on oltava oikeus eutanasiaan.',
 'Sukupuolen korjaamisen tulee olla mahdollista myös alle 18-vuotiaille.',
 'Viinit ja vahvat oluet pitää saada ruokakauppoihin.',
 'Energiajuomien myyminen alle 15-vuotiaille on kiellettävä. ',
 'Nuuskan, purutupakan ja nen

In [16]:
len(df.puolue.unique())

22

In [120]:
Q = df.loc[:, 'Suomen pitää olla edelläkävijä ilmastonmuutoksen vastaisessa taistelussa, vaikka se aiheuttaisi suomalaisille kustannuksia.':'On oikein nähdä vaivaa sen eteen, ettei vahingossakaan loukkaa toista.'].replace('-', np.nan)
Q = Q.apply(pd.to_numeric)
Q['puolue'] = df.puolue
Q = Q.dropna()

In [122]:
party_map = dict(zip(Q.puolue.unique(), list(range(len(Q.puolue.unique())))))
Q = Q.applymap(lambda x: party_map[x] if x in party_map else x)

In [125]:
m = KMeans(len(Q.puolue.unique()))
m.fit(Q[[i for i in Q.columns if i != 'puolue']])

KMeans(n_clusters=22)

In [127]:
accuracy = accuracy_score(Q.puolue, m.labels_)
print(accuracy)

0.02021276595744681


In [147]:
acc_list = list()
for i in range(1, 101):
    m2 = DBSCAN(eps=i)
    m2.fit(Q[[i for i in Q.columns if i != 'puolue']])
    acc_list.append(accuracy_score(Q.puolue, m2.labels_))
acc_list = list(enumerate(acc_list, start=1))

In [148]:
max(acc_list, key=lambda x: x[1])

(8, 0.09308510638297872)

In [57]:
a_dict = {
  "query": [
    {
      "code": "Tiedot",
      "selection": {
        "filter": "item",
        "values": [
          "cp"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat"
  }
}
a = requests.post("http://pxnet2.stat.fi/PXWeb/api/v1/fi/StatFin/kan/vtp/statfin_vtp_pxt_11sf.px", json=a_dict)
a_json = json.loads(a.content.decode('utf-8-sig'))

In [58]:
a_json['dataset'].keys()

dict_keys(['status', 'dimension', 'label', 'source', 'updated', 'value'])

In [51]:
pprint.pprint(requests.get("http://pxnet2.stat.fi/PXWeb/api/v1/fi/StatFin/kan/vtp/statfin_vtp_pxt_11sf.px").text)

('{"title":"Bruttokansantuote ja -tulo sekä tarjonta ja kysyntä, vuosittain '
 'muuttujina Taloustoimi, Vuosi ja '
 'Tiedot","variables":[{"code":"Taloustoimi","text":"Taloustoimi","values":["B1GMH","B1NMH","P1R","P2K","B1GPH","P51CK","D11K","D12K","B13N","P7R","P71R","P72R","P6K","P61K","P62K","P6K_M_P7R","D1R","D11R","D12R","D2N","D2R","D21R","D3K","D31K","D4_B4N","D4_B4NS11","D4_B4NS12","D4_B4NS13","D4_B4NS14","D4_B4NS15","D1_D4N","B11VV","B5G","B5N","B4N","B41N","B42N","B43N","B44N","D5_D7N","B6N","B6NS14","P3K","P31K","P31NCK","P32K","P3KS13","P3KS14_S15","P3KS15","F4S14","B8N","B8NS14","F4NS14","P51K","P51KS13","P51KXS13","P52K","P53K","DEB1G","B7NS14","P41K"],"valueTexts":["B1GMH '
 'Bruttokansantuote markkinahintaan","B1NMH Nettokansantuote '
 'markkinahintaan","P1R Tuotos perushintaan, tulona","P2K Välituotekäyttö, '
 'menona","B1GPH Bruttoarvonlisäys perushintaan","P51CK Kiinteän pääoman '
 'kuluminen, tulona","D11K Palkat ja palkkiot, menona","D12K Työnantajan '
 'sosiaalitu

In [112]:
from data_getter import SCB

In [113]:
s = SCB()

In [114]:
s.info()

AttributeError: 'function' object has no attribute 'get'